다음은 전처리 과정에서 분절을 수행하고, 다시 $분절 복원^{detokenization}$하는 과정을 설명한 것입니다.

다음과 같은 영어 원문이 주어졌다고 가정합니다

<br></br>
```
Natural Language Processing is one of biggest streams in artificial intelligence, and it becomes very popular after seq2seq's invention.
```
<br></br>

언어별 분절기 모듈 (ex. NLTK)을 통해 분절을 수행하고, 기존 띄어쓰기와 새롭게 분절기에 의해 만들어진 공백을 구분하기 위해 특수 문자 **"__"**를 원래 공백 위치에 삽입합니다. 다만 이 "__"문자는 기존의 _와 다른 문자로 특수 기호 입니다.

<br></br>
```
__Natural __Language __Processing __is __one __of __biggest __streams __in __artificial __intelligence , __and __it __becomes __very __popular __after __seq2seq 's __invention .
```
<br></br>

여기에 서브워드 단위 분절을 수행하며 이전 과정까지의 공백과 서브워드 단위 분절로 인한 공백을 구분하기 위해 특수 문자 "__"를 삽입합니다.

<br></br>
```
____Natural ____Language ____Processing ____is ____one ____of ____biggest ____streams ____in ____artificial ____intelligence __, ____and ____it ____becomes ____very ____popular ____after ____se q 2 se q __'s ____invention __.
```
<br></br>

이렇게 전처리 과정이 종료되었습니다. 이런 형태의 분절된 문장을 자연어 처리 모델에 훈련시키면 독같은 형태로 분절된 문장을 생성해낼 것입니다. 그럼 이 문장을 분절 복원하여 사람이 읽기 좋은 형태로 만들어 주어야 합니다.

먼저 공백을 제거합니다.

<br></br>
```
____Natural____Language____Processing____is____one____of____biggest____streams____in____artificial____intelligence__,____and____it____becomes____very____popular____after____seq2seq__'s____invention__.
```
<br></br>

그리고 특수 문자 "__"가 2개 동시에 잇는 문자열 "____"을 공백으로 치환합니다. 그럼 한개짜리 "__"만 남습니다.

<br></br>
```
Natural Language Processing is one of biggest streams in artificial intelligence__, and it becomes very popular after seq2seq__'s invention__.
```
<br></br>

마지막 남은 특수 문자 "__"를 제거합니다. 그럼 문장 복원이 완성됩니다.

<br></br>
```
Natural Language Processing is one of biggest streams in artificial intelligence, and it becomes very popular after seq2seq's invention.
```
<br></br>

### 분절 후처리

분절 이후에는 특수 문자를 분절 과정에서 새롭게 생긴 공백 다음에 삽입해야 합니다. 다음은 기존 공백과 전처리 단계에서 생성된 공백을 서로 구분하기 위한 특수 문자 "__"를 사입하는 파이썬 스크립트 예제입니다.

<br></br>
```python
import sys

STR = "__"

if __name__ == "__main__":
    ref_fn = sys.argv[1]
    
    f = open(ref_fn, "r")
    
    for ref in f:
        ref_tokens = ref.strip().split(" ")
        input_line = sys.stdin.readline().strip()

        if input_line != "":
            tokens = input_line.split(" ")

            idx = 0
            buf = []

            # We assum that stdin has more tokens than reference input
            for ref_token in ref_tokens:
                tmp_buf = []

                while idx < len(tokens):
                    if tokens[idx].strip() == "":
                        idx += 1
                        continue

                    tmp_buf += [tokens[idx]]
                    idx += 1

                    if "".join(tmp_buf) == ref_token:
                        break

                if len(tmp_buf) > 0:
                    buf += [STR + tmp_buf[0].strip()] + tmp_buf[1:]

            sys.stdout.write(" ".join(buf) + "\n")

        else:
            sys.stdout.write("\n")

    f.close()
```
<br></br>

이 스크립트의 사용 방법은 다음과 같습니다.

<br></br>
```bash
cat [before filename] | python tokenizer.py | python post_tokenize.py [before filename]
```
<br></br>

### 분절 복원 예제

앞서 설명한 분절 복원을 `sed` 명령어를 통해 수행할 경우의 예제입니다.

<br></br>
```bash
sed "s/ //g" | sed "s/ __/ /g" | sed "s/ _//g" | sed "s/%\s//g"
```
<br></br>

또는 다음의 파이썬 스크립트 예제처럼 처리할 수도 있습니다.

<br></br>
```python
import sys

if __name__ == "__main__":
    for line in sys.stdin:
        if line.strip() != "":
            if "__" in line:
                line = line.strip().replace(" ", "").replace("__", " ").replace("_", "").strip()
                
            else:
                line = line.strip().replace(" ", "").replace("_", " ").strip()
                
            sys.stdout.write(line + "\n")
            
        else:
            sys.stdout.write("\n")

```
<br></br>
